Imports

In [ ]:
from nb201 import NB201Benchmark
import numpy as np
import openai
from warmstart.utils_templates import FullTemplate
import ConfigSpace as CS
import google.generativeai as genai
from transformers import AutoTokenizer, AutoModelForCausalLM
import ollama

Load NB201 Benchmark

In [ ]:
b = NB201Benchmark(path="./nb201.pkl", dataset='cifar10')
cs = b.get_configuration_space()
config = cs.sample_configuration()  # samples a configuration uniformly at random

print(cs)
print("Numpy representation: ", config.get_array())
print("Dict representation: ", config.get_dictionary())

y, cost = b.objective_function(config)
print("Test error: %f %%" % y)
print("Runtime %f s" % cost)

Arguments for LLAMBO

In [ ]:
# task_context = {
#     'model': 'CNN',
#     'task': 'classification',
#     'tot_feats': 32 * 32 * 3,
#     'cat_feats': 0,
#     'num_feats': 32 * 32 * 3,
#     'n_classes': 10,
#     'metric': 'accuracy',
#     'lower_is_better': False,
#     'num_samples': 50000,
#     'hyperparameter_constraints': {
#         'max_depth': ['int', 'linear', [1, 15]],  # [type, transform, [min_value, max_value]]
#         'max_features': ['float', 'logit', [0.01, 0.99]],
#         'min_impurity_decrease': ['float', 'linear', [0.0, 0.5]],
#         'min_samples_leaf': ['float', 'logit', [0.01, 0.49]],
#         'min_samples_split': ['float', 'logit', [0.01, 0.99]],
#         'min_weight_fraction_leaf': ['float', 'logit', [0.01, 0.49]]
#     }
# }

task_context = {
    'model': 'RandomForest',
    'task': 'classification',
    'tot_feats': 30,
    'cat_feats': 0,
    'num_feat': 30,
    'n_classes': 2,
    'metric': 'accuracy',
    'lower_is_better': False,
    'num_samples': 455,
    'hyperparameter_constraints': {
        'max_depth': ['int', 'linear', [1, 15]],  # [type, transform, [min_value, max_value]]
        'max_features': ['float', 'logit', [0.01, 0.99]],
        'min_impurity_decrease': ['float', 'linear', [0.0, 0.5]],
        'min_samples_leaf': ['float', 'logit', [0.01, 0.49]],
        'min_samples_split': ['float', 'logit', [0.01, 0.99]],
        'min_weight_fraction_leaf': ['float', 'logit', [0.01, 0.49]]
    }
}

#me bo ni mapping t ints n operations n NAS
a = {
    "op_0_to_1": ['int', 'linear', [1, 15]],
    "op_0_to_2": 2,
    "op_0_to_3": 2,
    "op_1_to_2": 2,
    "op_1_to_3": 2,
    "op_2_to_3": 2,
}


#qetu me define warmstarting
def init_f():
    return


#should use the NB201 benchmark eval
def eval_point():
    return

Test gemma2b-it from huggingface

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("google/gemma-2b-it")
model = AutoModelForCausalLM.from_pretrained("google/gemma-2b-it").to("cuda")

In [ ]:
input_text = "Write me a poem about Machine Learning."
input_ids = tokenizer(input_text, return_tensors="pt").to("cuda")

outputs = model.generate(**input_ids, max_length=512)
print(tokenizer.decode(outputs[0]))

Ollama

In [ ]:
model = ollama.pull("llama3")


In [ ]:
response = ollama.chat(model="llama3", messages=[{'role': 'user', 'content': 'Why is the sky blue?'}])
print(response)
ollama.list()

Warmstart

In [ ]:
config = "No_Context"
metric = "acc"
exp_model = "RF"
NUM_INIT = 5
NUM_SEEDS = 10

template_object = FullTemplate(context=config, provide_ranges=True)

input_prompt = template_object.add_context(config_space=cs, num_recommendation=NUM_INIT, task_dict=task_context)
response = ollama.chat(model="llama3", messages=[{'role': 'user', 'content': input_prompt}])
print(response)